### We form the pre-trained extractive summary-embeddings from the training dataset.

In [1]:
import os
GPU_ID = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_ID)

import random as rd
import sys
import pandas as pd 
import numpy as np
import torch
import pickle

In [2]:
import params as pm 
import paths_args as ag
from recsys_lib20 import get_xs_bert_embeddings

In [3]:
# Random seed initialization
torch.manual_seed(pm.SEED_CONST)
torch.cuda.manual_seed(pm.SEED_CONST)
np.random.seed(pm.SEED_CONST)
rd.seed(pm.SEED_CONST)
torch.backends.cudnn.deterministic = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
split_csv_path = ag.CSV_PATH.format(pm.PREP_DOMAIN, "train")
print ("Training dataset will be read from ", split_csv_path, "\nGot it? Never forget!\n")

current_df = pd.read_csv(split_csv_path)

Training dataset will be read from  ../shared_datasets/patio_dataset/converted_csv/train.csv 
Got it? Never forget!



In [5]:
current_dataset = current_df.to_dict("records")
item_ids = current_df["item_id"].value_counts().index.tolist()
user_ids = current_df["user_id"].value_counts().index.tolist()

print ("Some quick sanity checks: ")
display (current_df)
print (len(user_ids))
print (type(user_ids))

ctr = 0
punc = [".", "!", "?"]

for ctr in range(len(current_dataset)):
    if (current_dataset[ctr]["reviews"][-1] not in punc):
        current_dataset[ctr]["reviews"] += "."

Some quick sanity checks: 


,user_id,item_id,ratings,reviews
0,652,229,4,This is a great Animal Repellent that uses Pep...
1,750,569,5,These are Christmas gifts for my sons I bought...
2,735,260,5,"I've used mostly peanut butter in mouse traps,..."
3,596,880,4,This hose is pretty neat! It's super lightweig...
4,1049,172,3,One of these days I'll find the perfect feeder...
...,...,...,...,...
10600,423,870,4,"This does as advertised, I almost always use m..."
10601,484,590,4,"Only bad thing is the grass catcher, dont use ..."
10602,238,126,3,I'm generally satisfied with this purchase - I...
10603,356,728,5,Was apprehensive though the video and descript...


1686
<class 'list'>


In [6]:
for rat in pm.RATIOS:

    out_user_pk_path = ag.OUT_PICKLE_PATH.format(pm.PREP_DOMAIN, pm.EMB_TYPE, "user", rat)
    out_item_pk_path = ag.OUT_PICKLE_PATH.format(pm.PREP_DOMAIN, pm.EMB_TYPE, "item", rat)

    print ("Pickled users will be saved here: ", out_user_pk_path, " @ ratio:", rat)
    print ("And, pickled items will be saved here: ", out_item_pk_path, " @ ratio:", rat) 

    print ("Getting (training) user embedding @ ratio:", rat)
    train_user_embs = get_xs_bert_embeddings(current_dataset, user_ids, "user_id", rat)

    with open(out_user_pk_path, "wb") as handle1:
        pickle.dump(train_user_embs, handle1)


    print ("Getting (training) item embedding @ ratio:", rat)
    train_item_embs = get_xs_bert_embeddings(current_dataset, item_ids, "item_id", rat)

    with open(out_item_pk_path, "wb") as handle2:
        pickle.dump(train_item_embs, handle2)

print ("\nFinished! Bye-bye!\n")

Pickled users will be saved here:  ../shared_datasets/patio_dataset/exsumm_embedding_mod_punc/user/ratio-0.4.pkl  @ ratio: 0.4
And, pickled items will be saved here:  ../shared_datasets/patio_dataset/exsumm_embedding_mod_punc/item/ratio-0.4.pkl  @ ratio: 0.4
Getting (training) user embedding @ ratio: 0.4
100%|██████████| 1686/1686 [46:30<00:00,  1.66s/it]


SystemExit: 